# Tigergraph<>Graphistry Fraud Demo: Raw REST

Accesses Tigergraph's fraud demo directly via manual REST calls

In [ ]:
#!pip install graphistry

import pandas as pd
import graphistry
import requests

#graphistry.register(key='MY_API_KEY', server='labs.graphistry.com', api=2)

TIGER = "http://MY_TIGER_SERVER:9000"

In [ ]:
#curl -X GET "http://MY_TIGER_SERVER:9000/query/circleDetection?srcId=111"

# string -> dict
def query_raw(query_string):
    url = TIGER + "/query/" + query_string
    r = requests.get(url)
    return r.json()


def flatten (lst_of_lst):
    try:
        if type(lst_of_lst[0]) == list:
            return [item for sublist in lst_of_lst for item in sublist]
        else:
            return lst_of_lst
    except:
        print('fail', lst_of_lst)
        return lst_of_lst

#str * dict -> dict
def named_edge_to_record(name, edge): 
    record = {k: edge[k] for k in edge.keys() if not (type(edge[k]) == dict) }
    record['type'] = name
    nested = [k for k in edge.keys() if type(edge[k]) == dict]
    if len(nested) == 1:
        for k in edge[nested[0]].keys():
            record[k] = edge[nested[0]][k]
    else:
        for prefix in nested:
            for k in edge[nested[prefix]].keys():
                record[prefix + "_" + k] = edge[nested[prefix]][k]
    return record


def query(query_string):
    results = query_raw(query_string)['results']
    out = {}
    for o in results:
        for k in o.keys():
            if type(o[k]) == list:
                out[k] = flatten(o[k])
    out = flatten([[named_edge_to_record(k,v) for v in out[k]] for k in out.keys()])
    print('# results', len(out))
    return pd.DataFrame(out)

  
def plot_edges(edges):
    return graphistry.bind(source='from_id', destination='to_id').edges(edges).plot()

# 1.  Fraud

## 1.a circleDetection

In [ ]:
circle = query("circleDetection?srcId=10")
circle.sample(3)

In [ ]:
plot_edges(circle)

## 1.b  fraudConnectivity

In [ ]:
connectivity = query("fraudConnectivity?inputUser=111&trustScore=0.1")
connectivity.sample(3)

In [ ]:
plot_edges(connectivity)

## Combined

In [ ]:
circle['provenance'] = 'circle'
connectivity['provenance'] = 'connectivity'

plot_edges(pd.concat([circle, connectivity]))

## Color by type

In [ ]:
edges = pd.concat([circle, connectivity])

froms = edges.rename(columns={'from_id': 'id', 'from_type': 'node_type'})[['id', 'node_type']]
tos = edges.rename(columns={'to_id': 'id', 'to_type': 'node_type'})[['id', 'node_type']]
nodes = pd.concat([froms, tos], ignore_index=True).drop_duplicates().dropna()
nodes.sample(3)

In [ ]:
nodes['node_type'].unique()

In [ ]:
#https://labs.graphistry.com/docs/docs/palette.html

type2color = {
    'User': 0,
    'Transaction': 1,
    'Payment_Instrument': 2,
    'Device_Token': 3   
}

nodes['color'] = nodes['node_type'].apply(lambda type_str: type2color[type_str])

nodes.sample(3)

In [ ]:
graphistry.bind(source='from_id', destination='to_id', node='id', point_color='color').edges(edges).nodes(nodes).plot()